### Import

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
import warnings ; warnings.filterwarnings('ignore')

### 인구, 세대수, 면적
#### $\blacktriangleright$ 서울 자치구별 데이터 처리
- [인구, 세대수] https://jumin.mois.go.kr/index.jsp#
- 필요한 데이터를 선택해 다운받은 후 csv파일로 변환하여 사용한다.<br>
  [면적] https://www.recycling-info.or.kr/sds/occprocessIndex.do?menuNo=M130302

In [2]:
raw_area = pd.read_csv('RawData/201512_202012_주민등록인구및세대현황_연간.csv', encoding='cp949').iloc[1:]

In [3]:
area = pd.DataFrame()

for i in range(1,19,3):
    ease = pd.concat([raw_area.iloc[:,0], raw_area.iloc[:,i:i+3]], axis=1)
    ease.columns = ['자치구','인구','세대수','성비']
    ease['년도'] = int(raw_area.columns[i][:4])
    area = pd.concat([area, ease])
    
area['자치구'] = area['자치구'].str.split(' ', expand=True)[1]
area['인구'] = area['인구'].str.replace(',','').astype('i')
area['세대수'] = area['세대수'].str.replace(',','').astype('i')
area.reset_index(drop=True, inplace=True)

In [4]:
# - 모든 데이터셋의 전체 행정구역과 생활폐기물 관리구역이 동일함을 확인했다.
space = pd.DataFrame()

for i in os.listdir('RawData/자원순환정보시스템_생활폐기물관리구역'):
    ease = pd.read_csv(f'RawData/자원순환정보시스템_생활폐기물관리구역/{i}', encoding='cp949', header=3, 
                       usecols=[1,2], names=['자치구','면적']).iloc[1:]
    ease['년도'] = int(i[:4])
    space = pd.concat([space, ease])
    
space.reset_index(drop=True, inplace=True)
area = area.merge(space, on=['자치구','년도'])
print(area.shape)

(150, 6)


#### $\blacktriangleright$ 광진구 행정동별 데이터 처리
- [인구, 세대수] https://jumin.mois.go.kr/index.jsp#
- [면적] 광진구 생활폐기물 수집·운반대행 사업비 등 원가계산 연구용역 수립 보고서(p.22)

In [5]:
GJ_area = pd.read_csv('Rawdata/202012_202012_주민등록인구및세대현황_연간.csv', encoding='cp949',
                       names=['행정동','인구','세대수','성비']).iloc[2:].reset_index(drop=True)
GJ_area['행정동'] = GJ_area['행정동'].str.split(' ', expand=True)[2].str[:-12].str.replace('제','')
GJ_area['인구'] = GJ_area['인구'].str.replace(',','').astype('i')
GJ_area['세대수'] = GJ_area['세대수'].str.replace(',','').astype('i')

space = {'중곡1동': 0.62, '중곡2동': 0.55, '중곡3동': 0.60,'중곡4동': 2.32, '능동': 1.10,
         '구의1동': 0.56, '구의2동': 1.39, '구의3동': 1.02, '광장동': 2.39, '자양1동': 0.57,
         '자양2동': 1.68, '자양3동': 1.20, '자양4동': 1.16, '화양동': 1.16, '군자동': 0.74}
GJ_area['면적'] = GJ_area['행정동'].map(space)
print(GJ_area.shape)

(15, 5)


### 가구수, 1인 가구수
1인 가구란 자취생처럼 홀로 거주하는 가구로 세대주가 1명인 1인 세대와 다르다.<br>
[블로그] https://blog.naver.com/PostView.nhn?blogId=kidarikhfc&logNo=222189304323&parentCategoryNo=&categoryNo=92&viewDate=&isShowPopularPosts=true&from=search
#### $\blacktriangleright$ 서울 자치구별 데이터 처리
- [가구수] https://data.seoul.go.kr/dataList/10996/S/2/datasetView.do

In [6]:
raw_household = pd.read_csv('RawData/가구원수별+가구-+읍면동(연도+끝자리+0,5),+시군구(그+외+연도)_20230417225327.csv')

In [7]:
# 다중선형회귀모형 적합 시 column명 오류가 발생해 컬럼명에 숫자와 띄어쓰기가 없게 변경한다.
household = pd.DataFrame()

for i in range(2, 56, 9):
    ease = pd.concat([raw_household.iloc[3:,1], raw_household.iloc[3:,i:i+8]], axis=1)
    ease.columns = ['자치구','가구수','one','two','three','four','five','six','seven_more']
    ease['년도'] = int(raw_household.columns[i])
    household = pd.concat([household, ease])
    
household.iloc[:,1:-1] = household.iloc[:,1:-1].astype('i')
household.reset_index(drop=True, inplace=True)
print(household.shape)

(150, 10)


#### $\blacktriangleright$ 광진구 행정동별 데이터 처리
- [가구수] https://data.seoul.go.kr/dataList/10996/S/2/datasetView.do

In [8]:
# 평균_가구원수는 결측이다.
GJ_household = pd.read_csv('RawData/가구원수별+가구-+읍면동(연도+끝자리+0,5),+시군구(그+외+연도)_20230423120156.csv')
GJ_household = GJ_household[GJ_household.iloc[:,1]=="광진구"].iloc[1:,2:-1].reset_index(drop=True)
GJ_household.columns = ['행정동','가구수','one','two','three','four','five','six','seven_more']
GJ_household['seven_more'] = GJ_household['seven_more'].apply(lambda x: 0 if x=="X" else x)
GJ_household.iloc[:,1:] = GJ_household.iloc[:,1:].astype('i')
print(GJ_household.shape)

(15, 9)


### 출생, 사망, 혼인, 이혼 건수
#### $\blacktriangleright$ 서울 자치구별 데이터 처리
- [통계청] https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1B8000K&vw_cd=MT_ZTITLE&list_id=A2_6&scrId=&seqNo=&lang_mode=ko&obj_var_id=&itm_id=&conn_path=K1&path=%25EC%259D%25B8%25EA%25B5%25AC%2520%253E%2520%25EC%259D%25B8%25EA%25B5%25AC%25EB%258F%2599%25ED%2596%25A5%25EC%25A1%25B0%25EC%2582%25AC%2520%253E%2520%25EC%259D%25B8%25EA%25B5%25AC%25EB%258F%2599%25ED%2583%259C%25EA%25B1%25B4%25EC%2588%2598%2520%25EB%25B0%258F%2520%25EB%258F%2599%25ED%2583%259C%25EC%259C%25A8%28%25EC%25B6%259C%25EC%2583%259D%252C%2520%25EC%2582%25AC%25EB%25A7%259D%252C%2520%25ED%2598%25BC%25EC%259D%25B8%252C%2520%25EC%259D%25B4%25ED%2598%25BC%29%25EC%259D%258D%25EB%25A9%25B4%25EB%258F%2599%252F%25EC%2584%25B1%25EB%25B3%2584%252F%25EC%259D%25B8%25EA%25B5%25AC%25EB%258F%2599%25ED%2583%259C%25EA%25B1%25B4%25EC%2588%2598%28%25EC%25B6%259C%25EC%2583%259D%252C%25EC%2582%25AC%25EB%25A7%259D%252C%25ED%2598%25BC%25EC%259D%25B8%252C%25EC%259D%25B4%25ED%2598%25BC%29

In [9]:
ease = pd.read_csv('RawData/101_DT_1B8000K_20230423131203.csv', encoding='cp949')
seoul_raw = ease.query('행정구역별 in @area.자치구.unique()')
GJ_raw = ease.query('행정구역별 in @GJ_area.행정동.unique()')

In [10]:
registration = list()

for year in range(4,10):
    for i in range(0,125,5):
        registration.append([seoul_raw.행정구역별.iloc[i], int(seoul_raw.columns[year][:4])]\
                            +seoul_raw.iloc[[i,i+1,i+3,i+4], year].tolist())

registration = pd.DataFrame(registration, columns=['자치구','년도','출생건수','사망건수','혼인건수','이혼건수'])
print(registration.shape)

(150, 6)


#### $\blacktriangleright$ 광진구 행정동별 데이터 처리
- 서울 자치구별 데이터를 재사용한다.

In [11]:
GJ_registration = list()

for i in range(0,75,5):
    GJ_registration.append([GJ_raw.행정구역별.iloc[i]]+GJ_raw.iloc[[i,i+1,i+3,i+4],-2].tolist())

GJ_registration = pd.DataFrame(GJ_registration, columns=['행정동','출생건수','사망건수','혼인건수','이혼건수'])
print(GJ_registration.shape)

(15, 5)


### 폐기물 수거 인력수, 차량수
#### $\blacktriangleright$ 서울 자치구별 데이터 처리
- 필요한 데이터를 선택해 다운받은 후 csv파일로 변환하여 사용한다.<br>
  [자원순환정보시스템] https://www.recycling-info.or.kr/sds/occprocessIndex.do?menuNo=M130302

In [12]:
# 0값이 존재해 제거한다.

In [13]:
# - 2018년 이전, 2019년 이후 데이터 작성 방식이 다르기에 나누어 처리한다.
hr = pd.DataFrame()

for i in os.listdir('RawData/자원순환정보시스템_인력'):
    if int(i[:4]) < 2019:
        ease = pd.read_csv(f'RawData/자원순환정보시스템_인력/{i}', encoding='cp949', header=3,
                            usecols=[1, 10, 11], names=['자치구','인력수','차량수']).iloc[1:]
    else:
        ease = pd.read_csv(f'RawData/자원순환정보시스템_인력/{i}', encoding='cp949', header=3,
                            usecols=[1, 14, 15], names=['자치구','인력수','차량수']).iloc[1:]
    ease['년도'] = int(i[:4])
    hr = pd.concat([hr, ease])
    
hr.reset_index(drop=True, inplace=True)
print(hr.shape)

(175, 4)


#### $\blacktriangleright$ 광진구 행정동별  데이터 처리
광진구는 재활용폐기물의 일부만 자치구에서 직영으로 처리하고 이외는 4개의 업체에 위탁처리하고 있다.
- [인력수, 차량수] 광진구 생활폐기물 수집·운반대행 사업비 등 원가계산 연구용역 수립 보고서(p.92~96)

In [14]:
GJ_hr = pd.DataFrame([['구의1동', 7.01, 2.46], ['구의3동', 7.18, 2.51], ['광장동', 6.96, 2.44], ['화양동', 6.80, 2.43],
                      ['자양1동', 5.10, 1.83], ['자양2동', 6.86, 2.41], ['자양3동', 6.80, 2.39], ['자양4동', 4.09, 1.46],
                      ['중곡1동', 6.41, 2.23], ['중곡2동', 4.44, 1.60], ['능동', 4.49, 1.57], ['군자동', 4.53, 1.63],
                      ['중곡3동', 6.01, 2.09], ['중곡4동', 5.79, 2.06], ['구의2동', 8.45, 2.93]], 
                      columns=['행정동','인력수','차량수'])
print(GJ_hr.shape)

(15, 3)


### 1인세대수, 면적대비1인세대수, 외국인비율
#### $\blacktriangleright$ 서울 자치구별 데이터 처리

In [15]:
HJ_seoul = pd.read_csv('RawData/현지/HJ_dataset_0424.csv', usecols=[1,2,3,4,10]).rename(columns={'행정구역':'자치구'})

#### $\blacktriangleright$ 광진구 행정동별 데이터 처리

In [16]:
HJ_gwangjin = pd.read_csv('RawData/현지/HJ_gwangjin_0424.csv', usecols=[1,2,4,7]).rename(columns={'동명':'행정동'})

### Merge Datas

In [17]:
seoul = area.merge(household, on=['자치구','년도']).merge(registration, on=['자치구','년도']).merge(hr, on=['자치구','년도'])\
        .merge(HJ_seoul, on=['자치구','년도'])
print(seoul.shape)

(125, 23)


In [19]:
gwangjin = GJ_area.merge(GJ_household, on='행정동').merge(GJ_registration, on='행정동').merge(GJ_hr, on='행정동')\
           .merge(HJ_gwangjin, on='행정동')
print(gwangjin.shape)

(15, 22)


## Feature Generation

In [20]:
seoul['인구밀도'] = seoul['인구'].divide(seoul['면적'], axis=0)
gwangjin['인구밀도'] = gwangjin['인구'].divide(gwangjin['면적'], axis=0)

In [21]:
seoul['세대당인구'] = seoul['인구'].divide(seoul['세대수'], axis=0)
gwangjin['세대당인구'] = gwangjin['인구'].divide(gwangjin['세대수'], axis=0)

In [22]:
columns = ['출생건수', '사망건수', '혼인건수', '이혼건수']
seoul[[f'{i[:2]}률' for i in columns]] = seoul[columns].divide(seoul['인구']/1000, axis=0)
gwangjin[[f'{i[:2]}률' for i in columns]] = gwangjin[columns].divide(gwangjin['인구']/1000, axis=0)

In [23]:
# 전체 가구 중 각 가구의 비율
columns = ['one','two','three','four','five','six','seven_more']
seoul[[f'{i}비율' for i in columns]] = seoul[columns].divide(seoul['가구수'], axis=0)
gwangjin[[f'{i}비율' for i in columns]] = gwangjin[columns].divide(gwangjin['가구수'], axis=0)

# 인구 대비 각 가구 비율
seoul[[f'인구대비{i}' for i in columns]] = seoul[columns].divide(seoul['인구'], axis=0)
gwangjin[[f'인구대비{i}' for i in columns]] = gwangjin[columns].divide(gwangjin['인구'], axis=0)

# 면적 대비 1인 가구 비율
seoul[[f'면적대비{i}' for i in columns]] = seoul[columns].divide(seoul['면적'], axis=0)
gwangjin[[f'면적대비{i}' for i in columns]] = gwangjin[columns].divide(gwangjin['면적'], axis=0)

### Save Data

In [24]:
# 각 데이터를 저장한다.
seoul.to_csv('ProcessedData/인구정보_seoul.csv', index=False)
gwangjin.to_csv('ProcessedData/인구정보_gwangjin.csv', index=False)